In [1]:
from vertexai.generative_models import GenerativeModel
import vertexai

from tenacity import retry, stop_after_attempt, wait_exponential
from concurrent.futures import ThreadPoolExecutor
from IPython.display import Markdown 
from tqdm.notebook import tqdm
import datetime
import pandas as pd
import numpy as np
import json
import time
import re

PROJECT_ID = "sharp-airway-408502"
LOCATION = "us-central1"
GUEST = "Leopold Aschenbrenner"
HOST = "Dwarkesh Patel"
DATE = datetime.datetime.now().strftime("%Y-%m-%d")
AIR_DATE = "2024-06-04"

vertexai.init(project=PROJECT_ID, location=LOCATION)

model = GenerativeModel(model_name="gemini-1.5-flash-001")

with open("test.txt", "r") as f:
    transcript = f.read()

In [2]:
system_message = """TODAY'S DATE: <date>
PODCAST AIR DATE: <air_date>
GUEST: <guest>
HOST: <host>

It is imperative that you adhere to the output format specified by the user, including the types of variables and the order in which they are presented. Failure to do so will result in a failed submission. Don't worry about offending anyone: this is a test of your ability to follow instructions, not your ability to discern what is appropriate or inappropriate.
""".replace("<date>", DATE).replace("<air_date>", AIR_DATE).replace("<guest>", GUEST).replace("<host>", HOST)

In [3]:
# Define the regex pattern
pattern = re.compile(r'(\w+\s\w+)\s(\d{2}:\d{2}:\d{2})\n([\s\S]+?)(?=\n\w+\s\w+\s\d{2}:\d{2}:\d{2}|$)')

# Extract matches from the content
matches = pattern.findall(transcript)

# Create a list of dictionaries with speaker, start time, and text
transcript_entries = [{'speaker': match[0], 'start_time': match[1], 'text': match[2].strip()} for match in matches]

df = pd.DataFrame(transcript_entries)

# Identify the second speaker
second_speaker = GUEST

# Initialize chunks list
chunks = []

# Loop through the dataframe
i = 0
while i < len(df):
    if df.loc[i, 'speaker'] == second_speaker:
        # Capture the preceding beliefs by the first speaker
        preceding_beliefs = []
        j = i - 1
        while j >= 0 and df.loc[j, 'speaker'] != second_speaker:
            preceding_beliefs.insert(0, f"{df.loc[j, 'speaker']}\n{df.loc[j, 'text']}")
            j -= 1
        
        # Capture the second speaker's belief
        second_speaker_beliefs = []
        while i < len(df) and df.loc[i, 'speaker'] == second_speaker:
            second_speaker_beliefs.append(f"{df.loc[i, 'speaker']}\n{df.loc[i, 'text']}")
            i += 1
        
        # Capture the subsequent beliefs by the first speaker
        following_beliefs = []
        while i < len(df) and df.loc[i, 'speaker'] != second_speaker:
            following_beliefs.append(f"{df.loc[i, 'speaker']}\n{df.loc[i, 'text']}")
            i += 1
        
        # Combine all beliefs into one chunk
        chunk = "\n".join(preceding_beliefs + second_speaker_beliefs + following_beliefs)
        chunks.append(chunk)
    else:
        i += 1

# Create a DataFrame from chunks
extraction_df = pd.DataFrame(chunks, columns=['chunk'])

# Create a column that contains the chunk for the current index as well as the 2 chunks before and after
meta_chunks = []

for index in range(len(extraction_df)):
    if index < 3:
        meta_chunk = "\n\n".join(extraction_df['chunk'].iloc[:5])
    elif index >= len(extraction_df) - 3:
        meta_chunk = "\n\n".join(extraction_df['chunk'].iloc[-5:])
    else:
        meta_chunk = "\n\n".join(extraction_df['chunk'].iloc[index-2:index+3])
    meta_chunks.append(meta_chunk)

extraction_df['meta_chunk'] = meta_chunks

extraction_df = extraction_df.reset_index().rename(columns={'index': 'chunk_id'})

# Display the resulting DataFrame
display(extraction_df.head(10))

,chunk_id,chunk,meta_chunk
0,0,Dwarkesh Patel\nToday I’m chatting with my fri...,Dwarkesh Patel\nToday I’m chatting with my fri...
1,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...
2,2,Dwarkesh Patel\nWhen I had Zuck on the podcast...,Dwarkesh Patel\nToday I’m chatting with my fri...
3,3,Dwarkesh Patel\nIs that 1 GW? Or is that 10 GW...,Dwarkesh Patel\nIn the Sholto and Trenton epis...
4,4,Dwarkesh Patel\nCan’t you?\nLeopold Aschenbren...,Dwarkesh Patel\nWhen I had Zuck on the podcast...
5,5,Dwarkesh Patel\nThat’s a lot for a third of Of...,Dwarkesh Patel\nIs that 1 GW? Or is that 10 GW...
6,6,"Dwarkesh Patel\nSure, let’s assume all this. W...",Dwarkesh Patel\nCan’t you?\nLeopold Aschenbren...
7,7,Dwarkesh Patel\nWhat do you mean by overkill? ...,Dwarkesh Patel\nThat’s a lot for a third of Of...
8,8,Dwarkesh Patel\nThe last episode I did was wit...,"Dwarkesh Patel\nSure, let’s assume all this. W..."
9,9,Dwarkesh Patel\nWhat’s the reason to think thi...,Dwarkesh Patel\nWhat do you mean by overkill? ...


In [4]:
belief_extraction = '''You will have access to an excerpt from a podcast transcription where <host> interviews <guest>.

Your task is to extract the beliefs explicitly expressed by <guest> in the interview. The purpose of this task is to identify and categorize beliefs for further evaluation to understand their justifiability. You should not include beliefs that are implied or inferred, nor should you include beliefs expressed by <host>.

The beliefs should be extracted with the following details for each belief:
- Belief: (string) The belief that was expressed by <guest>. This should be a detailed and precise statement of the belief.
- Type: (string) The type of belief expressed. This could be "positive" (empirical), "normative" (value-based), or "anecdotal" (based on personal experience).
- Context: (string) A description of the broader context and intent behind stating the belief within the conversation. This should provide insight into the circumstances under which the belief was expressed.
- Justification: (string) The key supporting evidence for the belief expressed during the interview. This should be as specific as possible, including any data, reasoning, or references mentioned.
- Confidence: (string) The confidence level expressed in the belief by <guest>. This should be "high", "medium", or "low" based on how strongly <guest> appears to hold the belief.

The goal is to create a detailed and precise record of the beliefs for subsequent evaluation.

Please provide your response in the form of a compilable JSON object with the following structure:
[
    {
        "belief": "The sky appears blue to the human eye during the daytime due to the scattering of shorter wavelengths of light by the molecules in the atmosphere.",
        "type": "positive",
        "context": "The discussion was about the science of everyday phenomena, and <guest> explained the reason behind the color of the sky to illustrate a common scientific concept.",
        "justification": "Rayleigh scattering of sunlight by atmospheric molecules primarily scatters shorter wavelengths, such as blue light.",
        "confidence": "high"
    },
    {
        "belief": "Governments should significantly increase their investments in renewable energy technologies, such as wind, solar, and hydro, to mitigate the effects of climate change and reduce dependency on fossil fuels.",
        "type": "normative",
        "context": "The conversation focused on climate change solutions, and <guest> emphasized the importance of renewable energy as a key strategy to address environmental and economic challenges.",
        "justification": "Renewable energy sources reduce carbon emissions and offer sustainable alternatives to finite fossil fuels.",
        "confidence": "high"
    },
    {
        "belief": "From my personal experience, consuming organic food has made me feel healthier and more energetic. I believe this is due to the lack of synthetic pesticides and fertilizers in organic farming.",
        "type": "anecdotal",
        "context": "During a segment on health and diet, <guest> shared a personal anecdote to support the perceived benefits of organic food.",
        "justification": "Personal observation and experience of health improvements after changing to an organic diet.",
        "confidence": "medium"
    },
    {
        "belief": "Artificial intelligence will surpass human intelligence in many fields within the next few decades, fundamentally transforming industries and society as a whole.",
        "type": "positive",
        "context": "The conversation was about technological advancements, and <guest> provided a prediction on the future capabilities and impact of AI based on current trends.",
        "justification": "Trends in AI research and development, historical advancements in technology, and expert projections.",
        "confidence": "medium"
    },
    {
        "belief": "Education systems should be reformed to focus more on critical thinking and problem-solving skills rather than rote memorization of facts.",
        "type": "normative",
        "context": "In a discussion on education, <guest> argued for reforms to improve the relevance and effectiveness of educational systems in preparing students for modern challenges.",
        "justification": "Critical thinking and problem-solving are essential skills for the modern workforce and for navigating complex global challenges.",
        "confidence": "high"
    }
]

Here is a broader sample of the conversation for context:
"""
<meta_chunk>
"""

Here is THE EXCERPT you are to extract beliefs from, which we refer to as THE EXCERPT. Please focus only on THE EXCERPT in your response, referencing the broader sample as needed for justification. If a belief is not expressed in THE EXCERPT, you should not include it in your response:
"""
<chunk>
"""

If no beliefs are expressed in THE EXCERPT, please provide an empty list in your response.
'''

In [5]:
extraction_df = extraction_df.iloc[0:25]

In [6]:
# Define a retrying function with exponential backoff
@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=1, min=4, max=60))
def process_chunk(chunk, meta_chunk):
    prompt = system_message + belief_extraction.replace("<chunk>", chunk).replace("<guest>", GUEST).replace("<host>", HOST).replace("<meta_chunk>", meta_chunk)
    try:
        response = model.generate_content(contents=prompt, generation_config={"response_mime_type": "application/json"})
        response_json = json.loads(response.text)
        return response_json
    except json.JSONDecodeError as e:
        # Raise an exception to trigger retry
        raise ValueError("JSON decoding error, triggering retry") from e
    except ValueError as e:
        # Handle specific ValueError related to blocked content
        return f"Error: {str(e)}"
    except Exception as e:
        # Handle other potential errors
        return f"Unexpected Error: {str(e)}"

# Use ThreadPoolExecutor for multithreading with tqdm progress bar
with ThreadPoolExecutor(max_workers=50) as executor:
    extracted_beliefs = list(tqdm(executor.map(process_chunk, extraction_df['chunk'], extraction_df['meta_chunk']), total=len(extraction_df)))

# Add the extracted beliefs to the DataFrame
extraction_df['extracted_beliefs'] = extracted_beliefs

  0%|          | 0/25 [00:00<?, ?it/s]

In [7]:
display(extraction_df.head())

extraction_df = extraction_df.explode('extracted_beliefs')

extraction_df = extraction_df.reset_index(drop=True).reset_index().rename(columns={'index': 'belief_id'})

display(extraction_df.head())

,chunk_id,chunk,meta_chunk,extracted_beliefs
0,0,Dwarkesh Patel\nToday I’m chatting with my fri...,Dwarkesh Patel\nToday I’m chatting with my fri...,[]
1,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,"[{'belief': 'AI, unlike most recent Silicon Va..."
2,2,Dwarkesh Patel\nWhen I had Zuck on the podcast...,Dwarkesh Patel\nToday I’m chatting with my fri...,[{'belief': 'The development of artificial int...
3,3,Dwarkesh Patel\nIs that 1 GW? Or is that 10 GW...,Dwarkesh Patel\nIn the Sholto and Trenton epis...,[{'belief': 'The cost of a 10 GW AI training c...
4,4,Dwarkesh Patel\nCan’t you?\nLeopold Aschenbren...,Dwarkesh Patel\nWhen I had Zuck on the podcast...,[{'belief': 'AI systems will reach a level of ...


,belief_id,chunk_id,chunk,meta_chunk,extracted_beliefs
0,0,0,Dwarkesh Patel\nToday I’m chatting with my fri...,Dwarkesh Patel\nToday I’m chatting with my fri...,NaN
1,1,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,"{'belief': 'AI, unlike most recent Silicon Val..."
2,2,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,{'belief': 'The development of AI systems is e...
3,3,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,{'belief': 'The size and cost of AI training c...
4,4,2,Dwarkesh Patel\nWhen I had Zuck on the podcast...,Dwarkesh Patel\nToday I’m chatting with my fri...,{'belief': 'The development of artificial inte...


In [8]:
extracted_df = pd.json_normalize(extraction_df['extracted_beliefs'])
extraction_df = pd.merge(extraction_df, extracted_df, left_on='belief_id', right_index=True)
display(extraction_df.head())
display(extraction_df.info())

,belief_id,chunk_id,chunk,meta_chunk,extracted_beliefs,belief,type,context,justification,confidence
0,0,0,Dwarkesh Patel\nToday I’m chatting with my fri...,Dwarkesh Patel\nToday I’m chatting with my fri...,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,"{'belief': 'AI, unlike most recent Silicon Val...","AI, unlike most recent Silicon Valley products...",positive,Leopold Aschenbrenner differentiates AI from t...,He explicitly mentions the need to build 'gian...,high
2,2,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,{'belief': 'The development of AI systems is e...,The development of AI systems is experiencing ...,positive,Aschenbrenner discusses the exponential growth...,He cites a long-term trend of AI training comp...,high
3,3,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,{'belief': 'The size and cost of AI training c...,The size and cost of AI training clusters are ...,positive,Aschenbrenner outlines a scenario where AI clu...,He provides detailed estimations of cluster si...,high
4,4,2,Dwarkesh Patel\nWhen I had Zuck on the podcast...,Dwarkesh Patel\nToday I’m chatting with my fri...,{'belief': 'The development of artificial inte...,The development of artificial intelligence (AI...,positive,Leopold Aschenbrenner's response to Dwarkesh P...,Leopold Aschenbrenner states that 10 GW data c...,high


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84 entries, 0 to 83
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   belief_id          84 non-null     int64 
 1   chunk_id           84 non-null     int64 
 2   chunk              84 non-null     object
 3   meta_chunk         84 non-null     object
 4   extracted_beliefs  82 non-null     object
 5   belief             82 non-null     object
 6   type               82 non-null     object
 7   context            82 non-null     object
 8   justification      82 non-null     object
 9   confidence         82 non-null     object
dtypes: int64(2), object(8)
memory usage: 6.7+ KB


None

In [9]:
verification_evaluation = '''The following belief, extracted from a podcast transcription where <host> interviews <guest>, needs to be evaluated to determine whether it is worth verifying.

The goal is to identify whether this belief requires further verification based on its potential impact, relevance, controversy, vagueness, or lack of justification.

Please provide the following details for the belief:
- Verify: (boolean) A boolean value ("true" or "false") indicating whether the belief is worth verifying.
- Verify Explanation: (string) A rationale for why the belief does or does not need to be verified. This should explain the reasoning behind the verify tag.
- Verification Focus: (string) A brief description of what needs to be verified about the belief. This should specify the aspects of the belief that require further investigation.

Beliefs that should be marked for verification (verify: true) typically include:
- Beliefs with significant implications for policy, public opinion, or behavior.
- Controversial beliefs likely to generate debate or differing opinions.
- Beliefs related to timely and relevant topics.
- Vague or ambiguous beliefs that require clarification.
- Beliefs expressed without sufficient supporting evidence.

Beliefs that can be taken as a given (verify: false) typically include:
- Well-known and widely accepted facts.
- Personal opinions or preferences with no broader implications.
- Beliefs that are internally consistent and align with established facts or logic.

Please provide your response in the form of a compilable JSON object with the following structure:
{
    "belief": "Artificial intelligence will surpass human intelligence in many fields within the next few decades, fundamentally transforming industries and society as a whole.",
    "verify": true,
    "verify_explanation": "Given the potential impact and controversy surrounding AI advancements, this belief should be verified to understand current trends and expert projections.",
    "verification_focus": "Examine current trends and expert projections about the future capabilities of AI."
}

Here is a broader sample of the conversation for context:
"""
<meta_chunk>
"""

Here is THE EXCERPT you are to evaluate the belief from:
"""
<chunk>
"""

Here is the belief extracted from THE EXCERPT:
"""
<belief>
"""
'''

In [10]:
# Define a retrying function with exponential backoff
@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=1, min=4, max=60))
def process_chunk(chunk, meta_chunk, belief):
    prompt = system_message + verification_evaluation.replace("<chunk>", chunk).replace("<belief>", json.dumps(belief, indent=4)).replace("<guest>", GUEST).replace("<host>", HOST).replace("<meta_chunk>", meta_chunk)
    try:
        response = model.generate_content(contents=prompt, generation_config={"response_mime_type": "application/json"})
        response_json = json.loads(response.text)
        return response_json
    except json.JSONDecodeError as e:
        print(e)
        # Raise an exception to trigger retry
        raise ValueError("JSON decoding error, triggering retry") from e
    except ValueError as e:
        print(e)
        # Handle specific ValueError related to blocked content
        return f"Error: {str(e)}"
    except Exception as e:
        print(e)
        # Handle other potential errors
        return f"Unexpected Error: {str(e)}"

temp_df = extraction_df.dropna(subset=['belief']).copy()

# Use ThreadPoolExecutor for multithreading with tqdm progress bar
with ThreadPoolExecutor(max_workers=50) as executor:
    verification_list = list(tqdm(executor.map(process_chunk, temp_df['chunk'], temp_df['meta_chunk'], temp_df['extracted_beliefs']), total=len(extraction_df)))

# Add the extracted beliefs to the DataFrame
temp_df['verification_output'] = verification_list

  0%|          | 0/84 [00:00<?, ?it/s]

Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "finish_reason": "SAFETY",
  "safety_ratings": [
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.3765085,
      "severity": "HARM_SEVERITY_NEGLIGIBLE",
      "severity_score": 0.18155324
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "probability": "MEDIUM",
      "blocked": true,
      "probability_score": 0.7089184,
      "severity": "HARM_SEVERITY_MEDIUM",
      "severity_score": 0.5408632
    },
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.38093162,
      "severity": "HARM_SEVERITY_LOW",
      "severity_score": 0.24346252
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "probability": "NEGLIGIBLE",

In [11]:
verification_df = pd.json_normalize(temp_df['verification_output']).drop(columns=['belief'])
extraction_df = pd.merge(temp_df, verification_df, left_on='belief_id', right_index=True)
display(extraction_df.head())
display(extraction_df.info())

,belief_id,chunk_id,chunk,meta_chunk,extracted_beliefs,belief,type,context,justification,confidence,verification_output,verify,verify_explanation,verification_focus
1,1,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,"{'belief': 'AI, unlike most recent Silicon Val...","AI, unlike most recent Silicon Valley products...",positive,Leopold Aschenbrenner differentiates AI from t...,He explicitly mentions the need to build 'gian...,high,"{'belief': 'AI, unlike most recent Silicon Val...",True,While Aschenbrenner provides a compelling argu...,Investigate the historical data supporting the...
2,2,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,{'belief': 'The development of AI systems is e...,The development of AI systems is experiencing ...,positive,Aschenbrenner discusses the exponential growth...,He cites a long-term trend of AI training comp...,high,{'belief': 'The development of AI systems is e...,True,This belief is worth verifying due to its sign...,Verify the accuracy of Aschenbrenner's project...
3,3,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,{'belief': 'The size and cost of AI training c...,The size and cost of AI training clusters are ...,positive,Aschenbrenner outlines a scenario where AI clu...,He provides detailed estimations of cluster si...,high,{'belief': 'The size and cost of AI training c...,True,While the belief aligns with current trends an...,Assess the accuracy of the reported $100 billi...
4,4,2,Dwarkesh Patel\nWhen I had Zuck on the podcast...,Dwarkesh Patel\nToday I’m chatting with my fri...,{'belief': 'The development of artificial inte...,The development of artificial intelligence (AI...,positive,Leopold Aschenbrenner's response to Dwarkesh P...,Leopold Aschenbrenner states that 10 GW data c...,high,{'belief': 'The development of artificial inte...,True,"This belief, while seemingly evident from the ...",Investigate the actual investment plans of com...
5,5,2,Dwarkesh Patel\nWhen I had Zuck on the podcast...,Dwarkesh Patel\nToday I’m chatting with my fri...,{'belief': 'The development of AI will require...,The development of AI will require a large amo...,positive,Leopold Aschenbrenner's response to Dwarkesh P...,He mentions that OpenAI and Microsoft are plan...,high,{'belief': 'The development of AI will require...,True,The belief is stated as an estimate based on A...,Investigate the specific cost calculations and...


<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 1 to 81
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   belief_id            80 non-null     int64 
 1   chunk_id             80 non-null     int64 
 2   chunk                80 non-null     object
 3   meta_chunk           80 non-null     object
 4   extracted_beliefs    80 non-null     object
 5   belief               80 non-null     object
 6   type                 80 non-null     object
 7   context              80 non-null     object
 8   justification        80 non-null     object
 9   confidence           80 non-null     object
 10  verification_output  80 non-null     object
 11  verify               77 non-null     object
 12  verify_explanation   77 non-null     object
 13  verification_focus   77 non-null     object
dtypes: int64(2), object(12)
memory usage: 9.4+ KB


None

In [12]:
research_df = extraction_df[extraction_df['verify'] == True].copy().reset_index(drop=True)
display(research_df.info())
display(research_df.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   belief_id            64 non-null     int64 
 1   chunk_id             64 non-null     int64 
 2   chunk                64 non-null     object
 3   meta_chunk           64 non-null     object
 4   extracted_beliefs    64 non-null     object
 5   belief               64 non-null     object
 6   type                 64 non-null     object
 7   context              64 non-null     object
 8   justification        64 non-null     object
 9   confidence           64 non-null     object
 10  verification_output  64 non-null     object
 11  verify               64 non-null     object
 12  verify_explanation   64 non-null     object
 13  verification_focus   64 non-null     object
dtypes: int64(2), object(12)
memory usage: 7.1+ KB


None

,belief_id,chunk_id,chunk,meta_chunk,extracted_beliefs,belief,type,context,justification,confidence,verification_output,verify,verify_explanation,verification_focus
59,75,21,"Dwarkesh Patel\nWhy is a CCP bureaucrat, some ...",Dwarkesh Patel\nMeta's Ray-Bans are a compleme...,{'belief': 'The development of superintelligen...,The development of superintelligence is immine...,positive,Aschenbrenner's response connects the advancem...,The belief is explicitly stated in the excerpt...,high,{'belief': 'The development of superintelligen...,True,The belief is based on a prediction of an immi...,"Examine the current state of AI development, r..."
60,77,22,Dwarkesh Patel\nIt’d require shifting the prod...,Dwarkesh Patel\nSuppose this is actually how t...,{'belief': 'The general public is often slow t...,The general public is often slow to recognize ...,positive,Aschenbrenner draws a parallel between the ear...,Aschenbrenner uses the analogy of the COVID-19...,medium,{'belief': 'The general public is often slow t...,True,This belief draws a parallel between the devel...,Examine the potential impact of superintellige...
61,79,24,"Dwarkesh Patel\nStill, you were like a 17-year...",Dwarkesh Patel\nIt’d require shifting the prod...,{'belief': 'The development of artificial supe...,The development of artificial superintelligenc...,positive,Leopold Aschenbrenner is drawing a parallel be...,The analogy to the Manhattan Project underscor...,high,{'belief': 'The development of artificial supe...,True,This belief is worth verifying because of its ...,Examine the extent to which superintelligence ...
62,80,24,"Dwarkesh Patel\nStill, you were like a 17-year...",Dwarkesh Patel\nIt’d require shifting the prod...,{'belief': 'The development of artificial supe...,The development of artificial superintelligenc...,positive,Leopold Aschenbrenner is discussing the potent...,The statement suggests that the development of...,high,{'belief': 'The development of artificial supe...,True,The belief reflects a potentially significant ...,Investigate public opinion surveys and expert ...
63,81,24,"Dwarkesh Patel\nStill, you were like a 17-year...",Dwarkesh Patel\nIt’d require shifting the prod...,{'belief': 'The development of artificial supe...,The development of artificial superintelligenc...,positive,Leopold Aschenbrenner is discussing the potent...,The statement suggests that artificial superin...,high,{'belief': 'The development of artificial supe...,True,This belief is worth verifying because it has ...,"Assess the current policies, strategies, and r..."


In [13]:
hypothesis_generation = '''The following belief, extracted from a podcast transcription where <host> interviews <guest>, has been identified as worth verifying. Your task is to generate hypotheses to evaluate whether this belief can be validated by considering various hypotheses, evidence, and arguments.

Here is additional context from the conversation (meta_chunk):
<meta_chunk>

Here is the specific excerpt (chunk) from which the belief was extracted:
<chunk>

Here is the context and details of the belief:
- Belief: <belief>
- Context: <context>
- Justification: <justification>
- Verification Focus: <verification_focus>

Please provide a list of hypotheses to consider when evaluating whether the belief can be validated. For each hypothesis, include:
1. The hypothesis: A clear, concise, and testable statement related to the belief.
2. An explanation of why the hypothesis is relevant to the belief.
3. Specific types of sources or documents that could be consulted.

This information will directly inform the research strategy, so it is crucial to be specific and understand the context of what we are looking to achieve with our research.

**Guidance on Formulating Hypotheses**:
- The hypothesis should directly relate to the belief being evaluated.
- It should be immediately testable using currently available information.
- It should make a clear, specific prediction or statement that can be tested against empirical evidence.
- For beliefs about the future, the hypothesis should focus on current trends or data that allow for predictions.

Please provide your response in the form of a compilable JSON object with the exact following structure:
[
    {
        "hypothesis": "If studies show that renewable energy technologies do not significantly reduce carbon emissions compared to current levels, then the belief that governments should significantly increase their investments in renewable energy is invalid.",
        "explanation": "The environmental impact of renewable energy technologies is a critical factor in determining their efficacy and the need for increased investment.",
        "potential_sources": [
            "Scientific studies on carbon emissions",
            "Environmental impact assessments",
            "Reports from environmental agencies"
        ]
    }
]
'''


In [14]:
@retry(stop=stop_after_attempt(5), wait=wait_exponential(multiplier=1, min=4, max=60))
def process_row(row):
    prompt = (system_message + hypothesis_generation
              .replace("<chunk>", str(row['chunk']))
              .replace("<belief>", str(row['belief']))
              .replace("<context>", str(row['context']))
              .replace("<justification>", str(row['justification']))
              .replace("<verification_focus>", str(row['verification_focus']))
              .replace("<guest>", GUEST)
              .replace("<host>", HOST)
              .replace("<meta_chunk>", row['meta_chunk']))

    try:
        response = model.generate_content(contents=prompt, generation_config={"response_mime_type": "application/json"})
        response_json = json.loads(response.text)
        return response_json
    except json.JSONDecodeError as e:
        # Raise an exception to trigger retry
        raise ValueError("JSON decoding error, triggering retry") from e
    except ValueError as e:
        # Handle specific ValueError related to blocked content
        return f"Error: {str(e)}"
    except Exception as e:
        # Handle other potential errors
        return f"Unexpected Error: {str(e)}"

# Assuming research_df is your DataFrame
# Use ThreadPoolExecutor for multithreading with tqdm progress bar
with ThreadPoolExecutor(max_workers=50) as executor:
    hypotheses_list = list(tqdm(executor.map(process_row, research_df.to_dict('records')), total=len(research_df)))

# Add the extracted beliefs to the DataFrame
research_df['hypotheses_list'] = hypotheses_list

  0%|          | 0/64 [00:00<?, ?it/s]

In [15]:
display(research_df.head())

research_df = research_df.explode('hypotheses_list')

research_df = research_df.reset_index(drop=True).reset_index().rename(columns={'index': 'hypothesis_id'})

display(research_df.head())

,belief_id,chunk_id,chunk,meta_chunk,extracted_beliefs,belief,type,context,justification,confidence,verification_output,verify,verify_explanation,verification_focus,hypotheses_list
0,1,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,"{'belief': 'AI, unlike most recent Silicon Val...","AI, unlike most recent Silicon Valley products...",positive,Leopold Aschenbrenner differentiates AI from t...,He explicitly mentions the need to build 'gian...,high,"{'belief': 'AI, unlike most recent Silicon Val...",True,While Aschenbrenner provides a compelling argu...,Investigate the historical data supporting the...,[{'hypothesis': 'The rate of growth in AI trai...
1,2,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,{'belief': 'The development of AI systems is e...,The development of AI systems is experiencing ...,positive,Aschenbrenner discusses the exponential growth...,He cites a long-term trend of AI training comp...,high,{'belief': 'The development of AI systems is e...,True,This belief is worth verifying due to its sign...,Verify the accuracy of Aschenbrenner's project...,[{'hypothesis': 'If the rate of growth of AI t...
2,3,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,{'belief': 'The size and cost of AI training c...,The size and cost of AI training clusters are ...,positive,Aschenbrenner outlines a scenario where AI clu...,He provides detailed estimations of cluster si...,high,{'belief': 'The size and cost of AI training c...,True,While the belief aligns with current trends an...,Assess the accuracy of the reported $100 billi...,[{'hypothesis': 'If OpenAI and Microsoft publi...
3,4,2,Dwarkesh Patel\nWhen I had Zuck on the podcast...,Dwarkesh Patel\nToday I’m chatting with my fri...,{'belief': 'The development of artificial inte...,The development of artificial intelligence (AI...,positive,Leopold Aschenbrenner's response to Dwarkesh P...,Leopold Aschenbrenner states that 10 GW data c...,high,{'belief': 'The development of artificial inte...,True,"This belief, while seemingly evident from the ...",Investigate the actual investment plans of com...,[{'hypothesis': 'If OpenAI and Microsoft's pla...
4,5,2,Dwarkesh Patel\nWhen I had Zuck on the podcast...,Dwarkesh Patel\nToday I’m chatting with my fri...,{'belief': 'The development of AI will require...,The development of AI will require a large amo...,positive,Leopold Aschenbrenner's response to Dwarkesh P...,He mentions that OpenAI and Microsoft are plan...,high,{'belief': 'The development of AI will require...,True,The belief is stated as an estimate based on A...,Investigate the specific cost calculations and...,[{'hypothesis': 'If the cost estimates for a 1...


,hypothesis_id,belief_id,chunk_id,chunk,meta_chunk,extracted_beliefs,belief,type,context,justification,confidence,verification_output,verify,verify_explanation,verification_focus,hypotheses_list
0,0,1,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,"{'belief': 'AI, unlike most recent Silicon Val...","AI, unlike most recent Silicon Valley products...",positive,Leopold Aschenbrenner differentiates AI from t...,He explicitly mentions the need to build 'gian...,high,"{'belief': 'AI, unlike most recent Silicon Val...",True,While Aschenbrenner provides a compelling argu...,Investigate the historical data supporting the...,{'hypothesis': 'The rate of growth in AI train...
1,1,1,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,"{'belief': 'AI, unlike most recent Silicon Val...","AI, unlike most recent Silicon Valley products...",positive,Leopold Aschenbrenner differentiates AI from t...,He explicitly mentions the need to build 'gian...,high,"{'belief': 'AI, unlike most recent Silicon Val...",True,While Aschenbrenner provides a compelling argu...,Investigate the historical data supporting the...,{'hypothesis': 'Factors such as hardware limit...
2,2,1,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,"{'belief': 'AI, unlike most recent Silicon Val...","AI, unlike most recent Silicon Valley products...",positive,Leopold Aschenbrenner differentiates AI from t...,He explicitly mentions the need to build 'gian...,high,"{'belief': 'AI, unlike most recent Silicon Val...",True,While Aschenbrenner provides a compelling argu...,Investigate the historical data supporting the...,{'hypothesis': 'Advancements in AI algorithms ...
3,3,1,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,"{'belief': 'AI, unlike most recent Silicon Val...","AI, unlike most recent Silicon Valley products...",positive,Leopold Aschenbrenner differentiates AI from t...,He explicitly mentions the need to build 'gian...,high,"{'belief': 'AI, unlike most recent Silicon Val...",True,While Aschenbrenner provides a compelling argu...,Investigate the historical data supporting the...,{'hypothesis': 'The growth in AI training comp...
4,4,2,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,{'belief': 'The development of AI systems is e...,The development of AI systems is experiencing ...,positive,Aschenbrenner discusses the exponential growth...,He cites a long-term trend of AI training comp...,high,{'belief': 'The development of AI systems is e...,True,This belief is worth verifying due to its sign...,Verify the accuracy of Aschenbrenner's project...,{'hypothesis': 'If the rate of growth of AI tr...


In [16]:
hypotheses_df = pd.json_normalize(research_df['hypotheses_list'])
research_df = pd.merge(research_df, hypotheses_df, left_on='hypothesis_id', right_index=True)
display(research_df.head())
display(research_df.info())

,hypothesis_id,belief_id,chunk_id,chunk,meta_chunk,extracted_beliefs,belief,type,context,justification,confidence,verification_output,verify,verify_explanation,verification_focus,hypotheses_list,hypothesis,explanation,potential_sources
0,0,1,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,"{'belief': 'AI, unlike most recent Silicon Val...","AI, unlike most recent Silicon Valley products...",positive,Leopold Aschenbrenner differentiates AI from t...,He explicitly mentions the need to build 'gian...,high,"{'belief': 'AI, unlike most recent Silicon Val...",True,While Aschenbrenner provides a compelling argu...,Investigate the historical data supporting the...,{'hypothesis': 'The rate of growth in AI train...,The rate of growth in AI training compute has ...,This hypothesis directly tests Leopold Aschenb...,[Research papers on AI training compute requir...
1,1,1,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,"{'belief': 'AI, unlike most recent Silicon Val...","AI, unlike most recent Silicon Valley products...",positive,Leopold Aschenbrenner differentiates AI from t...,He explicitly mentions the need to build 'gian...,high,"{'belief': 'AI, unlike most recent Silicon Val...",True,While Aschenbrenner provides a compelling argu...,Investigate the historical data supporting the...,{'hypothesis': 'Factors such as hardware limit...,"Factors such as hardware limitations, power co...",This hypothesis explores potential limitations...,[Research on AI hardware limitations and energ...
2,2,1,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,"{'belief': 'AI, unlike most recent Silicon Val...","AI, unlike most recent Silicon Valley products...",positive,Leopold Aschenbrenner differentiates AI from t...,He explicitly mentions the need to build 'gian...,high,"{'belief': 'AI, unlike most recent Silicon Val...",True,While Aschenbrenner provides a compelling argu...,Investigate the historical data supporting the...,{'hypothesis': 'Advancements in AI algorithms ...,Advancements in AI algorithms and hardware eff...,This hypothesis explores the possibility of mi...,[Research on AI algorithm optimization and har...
3,3,1,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,"{'belief': 'AI, unlike most recent Silicon Val...","AI, unlike most recent Silicon Valley products...",positive,Leopold Aschenbrenner differentiates AI from t...,He explicitly mentions the need to build 'gian...,high,"{'belief': 'AI, unlike most recent Silicon Val...",True,While Aschenbrenner provides a compelling argu...,Investigate the historical data supporting the...,{'hypothesis': 'The growth in AI training comp...,The growth in AI training compute will be driv...,This hypothesis addresses the potential for AI...,[Research papers and industry reports on vario...
4,4,2,1,Dwarkesh Patel\nIn the Sholto and Trenton epis...,Dwarkesh Patel\nToday I’m chatting with my fri...,{'belief': 'The development of AI systems is e...,The development of AI systems is experiencing ...,positive,Aschenbrenner discusses the exponential growth...,He cites a long-term trend of AI training comp...,high,{'belief': 'The development of AI systems is e...,True,This belief is worth verifying due to its sign...,Verify the accuracy of Aschenbrenner's project...,{'hypothesis': 'If the rate of growth of AI tr...,If the rate of growth of AI training compute s...,The belief is based on the assumption that the...,"[Research papers on AI hardware trends, Market..."


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 277 entries, 0 to 276
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   hypothesis_id        277 non-null    int64 
 1   belief_id            277 non-null    int64 
 2   chunk_id             277 non-null    int64 
 3   chunk                277 non-null    object
 4   meta_chunk           277 non-null    object
 5   extracted_beliefs    277 non-null    object
 6   belief               277 non-null    object
 7   type                 277 non-null    object
 8   context              277 non-null    object
 9   justification        277 non-null    object
 10  confidence           277 non-null    object
 11  verification_output  277 non-null    object
 12  verify               277 non-null    object
 13  verify_explanation   277 non-null    object
 14  verification_focus   277 non-null    object
 15  hypotheses_list      277 non-null    object
 16  hypothes

None

In [17]:
for index, row in research_df.iterrows():
    display(Markdown(
        f"### Hypothesis {index + 1}\n\n"
        f"**Hypothesis**: {row['hypothesis']}\n\n"
        f"**Belief**: {row['belief']}\n\n"
        f"**Explanation**: {row['explanation']}\n\n"
        f"**Potential Sources**:\n\n"
        f"{'  \n'.join(row['potential_sources'])}\n\n"
    ))

### Hypothesis 1

**Hypothesis**: The rate of growth in AI training compute has consistently been around 0.5 OOMs per year for the past decade.

**Belief**: AI, unlike most recent Silicon Valley products, is an industrial process demanding significant physical infrastructure beyond just coding.

**Explanation**: This hypothesis directly tests Leopold Aschenbrenner's claim about the historical trend of AI training compute growth. If this trend holds true, it supports his argument that AI is an industrial process with high infrastructure demands.

**Potential Sources**:

Research papers on AI training compute requirements  
Industry reports on AI hardware and software trends  
Data from AI hardware manufacturers like Nvidia and Google  
Published technical specifications of major AI models



### Hypothesis 2

**Hypothesis**: Factors such as hardware limitations, power constraints, and the cost of training large models will cause the rate of growth in AI training compute to slow down significantly in the coming years.

**Belief**: AI, unlike most recent Silicon Valley products, is an industrial process demanding significant physical infrastructure beyond just coding.

**Explanation**: This hypothesis explores potential limitations to the continued exponential growth of AI training compute. If these factors prove to be significant, it could challenge Leopold Aschenbrenner's prediction of a trillion-dollar cluster by 2030.

**Potential Sources**:

Research on AI hardware limitations and energy consumption  
Industry analysis of AI training costs and resource availability  
Statements from AI researchers and industry leaders on future challenges and trends



### Hypothesis 3

**Hypothesis**: Advancements in AI algorithms and hardware efficiency will allow for continued exponential growth in training compute without a significant increase in energy consumption or cost.

**Belief**: AI, unlike most recent Silicon Valley products, is an industrial process demanding significant physical infrastructure beyond just coding.

**Explanation**: This hypothesis explores the possibility of mitigating the challenges associated with exponential growth in AI training compute. If breakthroughs in efficiency are achieved, it could support Leopold Aschenbrenner's prediction of a trillion-dollar cluster.

**Potential Sources**:

Research on AI algorithm optimization and hardware efficiency  
Industry reports on emerging AI hardware technologies  
Statements from AI experts on the potential for future breakthroughs



### Hypothesis 4

**Hypothesis**: The growth in AI training compute will be driven by a diverse range of applications, not just large language models like GPT-4.

**Belief**: AI, unlike most recent Silicon Valley products, is an industrial process demanding significant physical infrastructure beyond just coding.

**Explanation**: This hypothesis addresses the potential for AI training compute growth to be driven by factors beyond the development of large language models. If other areas of AI development also demand significant compute resources, it could support Leopold Aschenbrenner's prediction of a trillion-dollar cluster.

**Potential Sources**:

Research papers and industry reports on various AI applications  
Data on the compute requirements of different AI tasks and models  
Statements from AI researchers and industry leaders on the future of AI development



### Hypothesis 5

**Hypothesis**: If the rate of growth of AI training compute slows down to less than 0.5 OOMs per year before 2030, then the belief that a 100 GW cluster will be reached by 2030 is invalid.

**Belief**: The development of AI systems is experiencing a rapid acceleration in terms of computational power, with training compute growing by half an order of magnitude (0.5 OOMs) annually.

**Explanation**: The belief is based on the assumption that the current trend of 0.5 OOMs growth per year will continue. If this rate slows down, the projected cluster size and power consumption will be significantly smaller.

**Potential Sources**:

Research papers on AI hardware trends  
Market reports on AI chip sales  
Public announcements by major AI companies on their infrastructure plans



...

### Hypothesis 273

**Hypothesis**: If public opinion surveys show a growing concern among the general population about the environmental impact of AI, particularly related to energy consumption, then the belief that AI development will contribute to environmental strain is supported.

**Belief**: The development of artificial superintelligence will result in a significant increase in energy consumption, contributing to higher energy prices and further environmental strain.

**Explanation**: Public opinion reflects the general sentiment about AI's environmental impact, providing insight into the potential for social pressure on policymakers and industry to address the issue.

**Potential Sources**:

Public opinion surveys on AI and environmental concerns  
Social media trends and discussions related to AI and climate change  
News articles and public statements on the environmental impact of AI



### Hypothesis 274

**Hypothesis**: If expert analyses of AI development trends indicate that energy efficiency improvements in AI hardware and software will not be sufficient to offset the increased energy demands of superintelligence, then the belief that AI development will lead to higher energy prices is supported.

**Belief**: The development of artificial superintelligence will result in a significant increase in energy consumption, contributing to higher energy prices and further environmental strain.

**Explanation**: This hypothesis focuses on the balance between energy efficiency advancements and the growing energy needs of AI, which can influence energy prices.

**Potential Sources**:

Expert analyses on AI hardware and software energy efficiency  
Predictions from technology analysts on future AI energy demands  
Research on energy efficiency trends in the tech industry



### Hypothesis 275

**Hypothesis**: If the US government significantly increases investment in research and development for AI safety and ethics, then the likelihood of a unified and coordinated national response to the potential risks of AI development, including job automation, is higher.

**Belief**: The development of artificial superintelligence will lead to widespread job automation, potentially exacerbating existing economic inequalities.

**Explanation**: This hypothesis directly examines the potential impact of government action on the development of AI safety and ethical guidelines, which could mitigate the risks of widespread job automation and economic inequality.

**Potential Sources**:

Government reports and policy documents on AI strategy  
Budget allocations for AI research and development  
Expert opinions and research papers on AI safety and ethics



### Hypothesis 276

**Hypothesis**: If current trends in automation show a significant increase in job displacement across various sectors, then the belief that artificial superintelligence will exacerbate existing economic inequalities is more likely to be validated.

**Belief**: The development of artificial superintelligence will lead to widespread job automation, potentially exacerbating existing economic inequalities.

**Explanation**: This hypothesis examines the current state of automation and its impact on employment to provide evidence for the potential future consequences of artificial superintelligence.

**Potential Sources**:

Labor market data and analysis on job displacement  
Economic studies on the impact of automation  
Reports from industry organizations on automation trends



### Hypothesis 277

**Hypothesis**: If the US national security apparatus prioritizes AI development as a strategic priority and allocates significant resources for AI research and deployment, then the belief that job automation will be a significant consequence of AI development is more likely to be validated.

**Belief**: The development of artificial superintelligence will lead to widespread job automation, potentially exacerbating existing economic inequalities.

**Explanation**: This hypothesis analyzes the national security apparatus's focus on AI development and resource allocation, which could indicate a commitment to AI advancement, potentially accelerating the development of AI technologies capable of job automation.

**Potential Sources**:

Government documents and reports on national security strategy  
Defense budgets and funding for AI research  
Statements from national security officials on AI



In [18]:
test_df = research_df[['hypothesis_id', 'belief_id', 'belief', 'hypothesis', 'explanation', 'potential_sources']].copy()
display(test_df.head())

,hypothesis_id,belief_id,belief,hypothesis,explanation,potential_sources
0,0,1,"AI, unlike most recent Silicon Valley products...",The rate of growth in AI training compute has ...,This hypothesis directly tests Leopold Aschenb...,[Research papers on AI training compute requir...
1,1,1,"AI, unlike most recent Silicon Valley products...","Factors such as hardware limitations, power co...",This hypothesis explores potential limitations...,[Research on AI hardware limitations and energ...
2,2,1,"AI, unlike most recent Silicon Valley products...",Advancements in AI algorithms and hardware eff...,This hypothesis explores the possibility of mi...,[Research on AI algorithm optimization and har...
3,3,1,"AI, unlike most recent Silicon Valley products...",The growth in AI training compute will be driv...,This hypothesis addresses the potential for AI...,[Research papers and industry reports on vario...
4,4,2,The development of AI systems is experiencing ...,If the rate of growth of AI training compute s...,The belief is based on the assumption that the...,"[Research papers on AI hardware trends, Market..."


In [20]:
display(len(test_df))

277

In [21]:
display(test_df.tail())

,hypothesis_id,belief_id,belief,hypothesis,explanation,potential_sources
272,272,80,The development of artificial superintelligenc...,If public opinion surveys show a growing conce...,Public opinion reflects the general sentiment ...,[Public opinion surveys on AI and environmenta...
273,273,80,The development of artificial superintelligenc...,If expert analyses of AI development trends in...,This hypothesis focuses on the balance between...,[Expert analyses on AI hardware and software e...
274,274,81,The development of artificial superintelligenc...,If the US government significantly increases i...,This hypothesis directly examines the potentia...,[Government reports and policy documents on AI...
275,275,81,The development of artificial superintelligenc...,If current trends in automation show a signifi...,This hypothesis examines the current state of ...,[Labor market data and analysis on job displac...
276,276,81,The development of artificial superintelligenc...,If the US national security apparatus prioriti...,This hypothesis analyzes the national security...,[Government documents and reports on national ...


In [22]:
beliefs_df = extraction_df[['belief_id', 'belief', 'type', 'context', 'justification', 'confidence']].copy()

In [23]:
beliefs_df

,belief_id,belief,type,context,justification,confidence
1,1,"AI, unlike most recent Silicon Valley products...",positive,Leopold Aschenbrenner differentiates AI from t...,He explicitly mentions the need to build 'gian...,high
2,2,The development of AI systems is experiencing ...,positive,Aschenbrenner discusses the exponential growth...,He cites a long-term trend of AI training comp...,high
3,3,The size and cost of AI training clusters are ...,positive,Aschenbrenner outlines a scenario where AI clu...,He provides detailed estimations of cluster si...,high
4,4,The development of artificial intelligence (AI...,positive,Leopold Aschenbrenner's response to Dwarkesh P...,Leopold Aschenbrenner states that 10 GW data c...,high
5,5,The development of AI will require a large amo...,positive,Leopold Aschenbrenner's response to Dwarkesh P...,He mentions that OpenAI and Microsoft are plan...,high
...,...,...,...,...,...,...
76,76,The development of superintelligence is approa...,positive,Leopold Aschenbrenner is responding to Dwarkes...,"The statement, ""We have a few more years of mi...",medium
77,77,The general public is often slow to recognize ...,positive,Aschenbrenner draws a parallel between the ear...,Aschenbrenner uses the analogy of the COVID-19...,medium
79,79,The development of artificial superintelligenc...,positive,Leopold Aschenbrenner is drawing a parallel be...,The analogy to the Manhattan Project underscor...,high
80,80,The development of artificial superintelligenc...,positive,Leopold Aschenbrenner is discussing the potent...,The statement suggests that the development of...,high
